In [1]:
from keras.models import model_from_yaml
import  numpy as np


Using gpu device 0: GeForce GTX 960


In [120]:
from vectorizer import Vectorizer
py="\n\nfrom keras.models import Sequential\nfrom keras.layers.recurrent import LSTM, JZS1, GRU\n" * 20


In [121]:
vec = Vectorizer(py, cutoff=60)

print(dict(enumerate(vec.dictionary)))

{0: '', 1: 'r', 2: 'e', 3: ' ', 4: 'm', 5: 'o', 6: 'a', 7: 's', 8: 't', 9: '\n', 10: 'S', 11: 'i', 12: 'l', 13: '.', 14: 'k', 15: 'u', 16: ',', 17: 'f', 18: 'p', 19: 'n', 20: '1', 21: 'J', 22: 'G', 23: 'Z', 24: 'c', 25: 'T', 26: 'U', 27: 'y', 28: 'd', 29: 'q', 30: 'L', 31: 'M', 32: 'R'}


In [123]:
window_len = 40
matrix = vec.to_matrix(py)
trainX = []
trainy = []

for i in range(len(matrix)-window_len-1):
    trainX.append(matrix[i:i+window_len])
    trainy.append(matrix[i+window_len]) 

trainX = np.array(trainX,dtype="float32")
trainy = np.array(trainy,dtype="float32")

print(trainX.shape)
print(trainy.shape)


(1719, 40, 33)
(1719, 33)


In [141]:

from keras.models import Sequential
from keras.optimizers import Adadelta, SGD, Adagrad, RMSprop
from keras.layers.core import Dense, TimeDistributedDense, Flatten
from keras.layers.recurrent import LSTM, JZS1, GRU

lstmmodel = Sequential()
lstmmodel.add(LSTM(33,60, return_sequences=True))
lstmmodel.add(LSTM(60,60))
lstmmodel.add(Dense(60,33))
lstmmodel.compile(optimizer=SGD(lr=0.1,momentum=0.8),loss='mse',class_mode="categorical")

print("Initial loss")
print(lstmmodel.evaluate(trainX,trainy,show_accuracy=True))
print("Training")
lstmmodel.fit(trainX,trainy,nb_epoch = 100, show_accuracy=True,verbose=1)
print("New loss")
print(lstmmodel.evaluate(trainX,trainy,show_accuracy=True))

Initial loss
1719/1719 [==============================] - 1s     
[0.044052711637596394, 0.0]
Training
Epoch 0
1719/1719 [==============================] - 6s - loss: 0.0355 - acc: 0.0355     
Epoch 1
1719/1719 [==============================] - 6s - loss: 0.0310 - acc: 0.0913     
Epoch 2
1719/1719 [==============================] - 6s - loss: 0.0300 - acc: 0.1070     
Epoch 3
1719/1719 [==============================] - 6s - loss: 0.0295 - acc: 0.1193     
Epoch 4
1719/1719 [==============================] - 6s - loss: 0.0293 - acc: 0.1280     
Epoch 5
1719/1719 [==============================] - 6s - loss: 0.0290 - acc: 0.1326     
Epoch 6
1719/1719 [==============================] - 6s - loss: 0.0288 - acc: 0.1344     
Epoch 7
1719/1719 [==============================] - 6s - loss: 0.0287 - acc: 0.1373     
Epoch 8
1719/1719 [==============================] - 6s - loss: 0.0285 - acc: 0.1559     
Epoch 9
1719/1719 [==============================] - 6s - loss: 0.0284 - acc: 0.1542   

In [145]:
print(lstmmodel.evaluate(trainX,trainy,show_accuracy=True))

1719/1719 [==============================] - 1s     
[0.023333431469138934, 0.51018033740546831]


In [146]:
lstmyaml=lstmmodel.to_yaml()
print(lstmyaml)

class_mode: categorical
layers:
- {activation: tanh, forget_bias_init: one, init: glorot_uniform, inner_activation: hard_sigmoid,
  inner_init: orthogonal, input_dim: 33, name: LSTM, output_dim: 60, return_sequences: true,
  truncate_gradient: -1}
- {activation: tanh, forget_bias_init: one, init: glorot_uniform, inner_activation: hard_sigmoid,
  inner_init: orthogonal, input_dim: 60, name: LSTM, output_dim: 60, return_sequences: false,
  truncate_gradient: -1}
- {W_constraint: null, W_regularizer: null, activation: linear, activity_regularizer: null,
  b_constraint: null, b_regularizer: null, init: glorot_uniform, input_dim: 60, name: Dense,
  output_dim: 33}
loss: mean_squared_error
name: Sequential
optimizer: {decay: 0.0, lr: 0.1, momentum: 0.8, name: SGD, nesterov: false}
theano_mode: null



## Let's copy the LSTM model into a new model and transfer the weights

In [152]:
lstmcopy = model_from_yaml(lstmyaml)
print(lstmcopy.evaluate(trainX,trainy,show_accuracy=True))
lstmweights=lstmmodel.get_weights()
lstmcopy.set_weights(lstmweights)
print("Original:", lstmmodel.evaluate(trainX,trainy,show_accuracy=True))
print("Copy:",lstmcopy.evaluate(trainX,trainy,show_accuracy=True))


1719/1719 [==============================] - 1s     
[0.043840713632131052, 0.022687609075043629]
1719/1719 [==============================] - 1s     
Original: [0.023333431469138934, 0.51018033740546831]
1719/1719 [==============================] - 1s     
Copy: [0.023333431469138934, 0.51018033740546831]


## So it works with LSTM!
### Trained accuracy 51%, copied accuracy 51%


## Let's try it with JZS1 then!

In [153]:
jzs1model = Sequential()
jzs1model.add(JZS1(33,60, return_sequences=True))
jzs1model.add(JZS1(60,60))
jzs1model.add(Dense(60,33))
jzs1model.compile(optimizer=SGD(lr=0.1,momentum=0.8),loss='mse',class_mode="categorical")

DEBUG: nvcc STDOUT mod.cu
   Creating library D:/Users/nico/AppData/Local/Theano/compiledir_Windows-7-6.1.7601-SP1-AMD64_Family_21_Model_2_Stepping_0_AuthenticAMD-3.4.3-64/tmpopwc0glz/m2f8b298e743455e6e0d4423a9bb3d736.lib and object D:/Users/nico/AppData/Local/Theano/compiledir_Windows-7-6.1.7601-SP1-AMD64_Family_21_Model_2_Stepping_0_AuthenticAMD-3.4.3-64/tmpopwc0glz/m2f8b298e743455e6e0d4423a9bb3d736.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library D:/Users/nico/AppData/Local/Theano/compiledir_Windows-7-6.1.7601-SP1-AMD64_Family_21_Model_2_Stepping_0_AuthenticAMD-3.4.3-64/tmppk2c_sim/me3622db1900040e92072ee3096913150.lib and object D:/Users/nico/AppData/Local/Theano/compiledir_Windows-7-6.1.7601-SP1-AMD64_Family_21_Model_2_Stepping_0_AuthenticAMD-3.4.3-64/tmppk2c_sim/me3622db1900040e92072ee3096913150.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library D:/Users/nico/AppData/Local/Theano/compiledir_Windows-7-6.1.7601-SP1-AMD64_Family_21_Model_2_Stepping_0_AuthenticAMD-3.4.3-64/tmppjjej

In [154]:
print("Initial loss")
print(jzs1model.evaluate(trainX,trainy,show_accuracy=True))
print("Training")
import time
start=time.time()
jzs1model.fit(trainX,trainy,nb_epoch = 5, show_accuracy=True,verbose=1)
print("Turning verbose off and training 100 epochs, this will take approx.", 20*(time.time()-start),"seconds")
from sys import stdout
stdout.flush()
jzs1model.fit(trainX,trainy,nb_epoch = 100, show_accuracy=True,verbose=0)
print("New loss")
print(jzs1model.evaluate(trainX,trainy,show_accuracy=True))

Initial loss
1719/1719 [==============================] - 1s     
[0.042056869431558633, 0.011634671320535195]
Training
Epoch 0
1719/1719 [==============================] - 3s - loss: 0.0392 - acc: 0.0145     
Epoch 1
1719/1719 [==============================] - 3s - loss: 0.0365 - acc: 0.0425     
Epoch 2
1719/1719 [==============================] - 3s - loss: 0.0356 - acc: 0.0762     
Epoch 3
1719/1719 [==============================] - 3s - loss: 0.0350 - acc: 0.0931     
Epoch 4
1719/1719 [==============================] - 3s - loss: 0.0344 - acc: 0.0989     
Turning verbose off and training 100 epochs, this will take approx. 398.0827760696411 seconds
New loss
1719/1719 [==============================] - 0s     
[0.023093559178390982, 0.56719022687609078]


In [157]:
jzs1yaml=jzs1model.to_yaml()
print(jzs1yaml)

class_mode: categorical
layers:
- {activation: tanh, init: glorot_uniform, inner_activation: sigmoid, inner_init: orthogonal,
  input_dim: 33, name: JZS1, output_dim: 60, return_sequences: true, truncate_gradient: -1}
- {activation: tanh, init: glorot_uniform, inner_activation: sigmoid, inner_init: orthogonal,
  input_dim: 60, name: JZS1, output_dim: 60, return_sequences: false, truncate_gradient: -1}
- {W_constraint: null, W_regularizer: null, activation: linear, activity_regularizer: null,
  b_constraint: null, b_regularizer: null, init: glorot_uniform, input_dim: 60, name: Dense,
  output_dim: 33}
loss: mean_squared_error
name: Sequential
optimizer: {decay: 0.0, lr: 0.1, momentum: 0.8, name: SGD, nesterov: false}
theano_mode: null



In [158]:
jzs1copy = model_from_yaml(jzs1yaml)
print("Untrained:",jzs1copy.evaluate(trainX,trainy,show_accuracy=True))
jzs1weights = jzs1model.get_weights()
jzs1copy.set_weights(jzs1weights)
print("Original:", jzs1model.evaluate(trainX,trainy,show_accuracy=True))
print("Copy:",jzs1copy.evaluate(trainX,trainy,show_accuracy=True))

1719/1719 [==============================] - 0s     
Untrained: [0.045702425345839147, 0.022687609075043629]
1719/1719 [==============================] - 0s     
Original: [0.023093559178390982, 0.56719022687609078]
1719/1719 [==============================] - 0s     
Copy: [0.040583413824603752, 0.056428155904595698]


## Unlike with LSTM, with JZS1 the progress is mostly lost:
### Untrained: [0.045702425345839147, 0.022687609075043629]
### Copy: [0.040583413824603752, 0.056428155904595698]
### Accuracy 2.2% -> 5.6% , trained accuracy was 56.7%
